In [1]:
# Reminder: a function can get a function as parameter:

def high_order_func(func: callable, x: int) -> int:
    return func(x)+x

func = lambda a : a**2
print(type(func))
print(high_order_func(func, 2))

<class 'function'>
6


In [2]:
# Reminder: a function can return a function:
def glob_f():
    print("I'm in the glob function")
def out_f():
    return glob_f
print(out_f)
print(out_f())
f = out_f()
print(type(f))
f()
out_f()()

<function out_f at 0x0000027DC270B2E0>
<function glob_f at 0x0000027DC270B400>
<class 'function'>
I'm in the glob function
I'm in the glob function


In [3]:
# Reminder: a function can return a function:

def out_f():
    def in_f():
        print("I'm in the inner function")
    return in_f
print(out_f)
print(out_f())
f = out_f()
print(type(f))
f()
out_f()()

<function out_f at 0x0000027DC270B490>
<function out_f.<locals>.in_f at 0x0000027DC270B370>
<class 'function'>
I'm in the inner function
I'm in the inner function


In [4]:
# A function both accept a function parameter, and return a function:

def wrap(func):
    def wrapper():
        print(f"starting {func.__name__}")
        func()
        print(f"ending {func.__name__}")
    return wrapper

In [5]:
def a_function():
    print("I'm a function")

a_function()

print("\nFirst way: variable ")
a_wrapped_function = wrap(a_function)
a_wrapped_function()

print("\nSecond way: outer_function(a_function)() ")
wrap(a_function)()

I'm a function

First way: variable 
starting a_function
I'm a function
ending a_function

Second way: outer_function(a_function)() 
starting a_function
I'm a function
ending a_function


### A function that accepts a function argument and returns a function is called a "decorator".

In [6]:
@wrap
def a_function():
    print("I'm a new function")

# Equivalent to:
#    def a_function():
#       print("I'm a new function")
#    a_function = wrap(a_function)

a_function()

@wrap
def another_function():
    print("I'm another function")

print()
another_function()


starting a_function
I'm a new function
ending a_function

starting another_function
I'm another function
ending another_function


In [7]:
@wrap
def param_function(num: int):
    print(f"{num} is a number")
param_function(3) #We should get a TypeError
# param_function() #We should get another TypeError

TypeError: wrap.<locals>.wrapper() takes 0 positional arguments but 1 was given

In [8]:
# Reminder: *args

def my_sum(*args):
    print(args)
    print(type(args))
    result = 0
    for x in args:
        result += x
    return result

print(my_sum(1,2,3,4,5))

(1, 2, 3, 4, 5)
<class 'tuple'>
15


In [9]:
# Reminder: **kwargs

def f(**kwargs):
    print(kwargs)
    print(type(kwargs))
    for key, val in kwargs.items():
        print(key, '->', val)

f(e=1, r=2, j=3)

{'e': 1, 'r': 2, 'j': 3}
<class 'dict'>
e -> 1
r -> 2
j -> 3


In [10]:
def wrap_with_params(func):
    def wrapper(*args , **kwargs):
        print(f"starting {func.__name__}")
        func(*args, **kwargs)
        print(f"ending {func.__name__}")
    return wrapper

@wrap_with_params
def param_function(num: int):
    print(f"{num} is a number")
param_function(3)
print()
param_function(num=3)

starting param_function
3 is a number
ending param_function

starting param_function
3 is a number
ending param_function


In [11]:
def wrap_with_return(func):
    def wrapper(*args , **kwargs):
        print(f"starting {func.__name__}")
        return_val = func(*args, **kwargs)
        print(f"ending {func.__name__}")
        return return_val
    return wrapper

@wrap_with_return
def change_name_to_upper():
    """Changes the first character of the name to upper case """
    first_name= input("Your first name is: ")
    last_name= input("Your last name is: ")
    fname_list=list(first_name)
    lname_list=list(last_name)
    fname_list[0]= fname_list[0].upper()
    lname_list[0]= lname_list[0].upper()
    return "".join(fname_list)+" "+"".join(lname_list)
print(change_name_to_upper())

starting change_name_to_upper


IndexError: list index out of range

In [7]:
@wrap_with_return
def param_function(num: int):
    print(f"{num} is a number")

param_function(2)

starting param_function
2 is a number
ending param_function


## Putting two wrappers on the same function

In [8]:

def my_timer(orig_func):
    import time
    def wrapper(*args, **kwargs):
        time_before = time.time()
        result = orig_func(*args, **kwargs)
        time_after = time.time()
        print(f'my_timer: {orig_func.__name__} ran in: {time_after-time_before} sec')
        return result

    return wrapper

In [9]:
@my_timer
def test2(a, b):
    print(f'{a}+{b}={a+b}')
    for i in range(100000000): j=i*i # 5 sec

test2(5, 7)

5+7=12
my_timer: test2 ran in: 5.524379730224609 sec


In [12]:
@wrap_with_return
@my_timer
def test3(a, b):
    print(f'{a}+{b}={a+b}')

test3(5, 7)  # Wrong result: "starting wrapper... ending wrapper"

starting wrapper
5+7=12
my_timer: test3 ran in: 0.0 sec
ending wrapper


In [13]:
from functools import wraps

def my_logger(orig_func):
    @wraps(orig_func)
    def wrapper(*args , **kwargs):
        print(f"starting {orig_func.__name__}")
        return_val = orig_func(*args, **kwargs)
        print(f"ending {orig_func.__name__}")
        return return_val
    return wrapper

def my_timer(orig_func):
    import time
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print(f'{orig_func.__name__} ran in: {t2} sec')
        return result
    return wrapper


@my_logger
@my_timer
def test4(a, b):
    print(f'{a}+{b}={a+b}')

test4(5, 7)

starting test4
5+7=12
test4 ran in: 0.0 sec
ending test4
